# __Training__

* ✅ **Sub-Modules**
    * ✅ Discriminator
    * ✅ Classifier
    * ✅ Phase Regressor
    * ✅ Amplitude Regressor


* ✅ **Transfer Learning**
    * ✅ Transfer 1 (Direct) 
        * ✅ Train Transfer Head on Trace, Phase and Amplitude
        * ✅ Fine-Tune whole Model
    * ✅ Transfer 2 (Long) 
        * ✅ Train Transfer Head on Trace
        * ✅ Fine Tune Phase
        * ✅ Fine Tune Amplitude


* ✅ **Sequential Trianing** 
    * ✅ Train whole model on trace
    * ✅ Train whole model on phase
    * ✅ Train whole model on amplitude


* ✅ **Control Training** 
    * ✅ Train whole model on trace, phase and amplitude
    


In [1]:
try:
    from google.colab import drive
    drive.mount('/content/drive')   
    RUNNING_IN_COLAB = True
    
except:
    print("Not in Colab")
    RUNNING_IN_COLAB = False
    
if RUNNING_IN_COLAB:
    print("Running in Colab")
    #pull git repo and install requirements
    !git clone https://github.com/tmengel/Deep-Learning.git
    %cd Deep-Learning/finalProject
    %pip install uproot
    %pip install pandas

import pulsenet as pn
import pandas as pd  
import tensorflow as tf
from tensorflow import keras
from keras import layers, models

Not in Colab


2023-05-14 01:31:00.586402: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-14 01:31:00.724375: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/tmengel/root/lib
2023-05-14 01:31:00.724397: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-14 01:31:01.546409: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared obj

In [2]:
filename = "ysoTracesWithPileup.root"
if RUNNING_IN_COLAB:
    filename = "/content/drive/MyDrive/DeepLearningFinalProject/ysoTracesWithPileup.root"
    
def GetData():
    x_trace, y_trace, y_phase, y_amp = pn.CreateData(filename, pileup_split=0.5, phase_min=0.1, phase_max=20, amplitude_min=0.5, amplitude_max=1.5)
    y_pileup = pn.EncodePileup(y_phase)
    return x_trace, y_trace, y_phase, y_amp, y_pileup

epochs = 500
batchsize = 128 
validation_split = 0.2

def SaveModel(disriminator_base, classifier_base, phase, amplitude, tracenet, modelfile):
    
    input = layers.Input(shape=(1, 300))  # Returns a placeholder tensor
    classifer_feature_vec = pn.TraceClassifierBase(name = "classifier_base")(input)
    discriminator_feature_vec = pn.TraceDiscriminatorBase(name = "discriminator_base")(input)
    trace_output = pn.TraceClassifierDiscriminatorHead(name = "tracenet")([discriminator_feature_vec, classifer_feature_vec])
    phase_output = pn.TracePhaseRegressor(name="phase")(trace_output)
    amplitude_output = pn.TraceAmplitudeRegressor(name ="amplitude")(trace_output)
    
    model = models.Model(inputs=input, outputs=[trace_output, phase_output, amplitude_output])
    model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
    model_names = [model.layers[i].name for i in range(len(model.layers))]
    # load weights
    model.layers[model_names.index("discriminator_base")].load_weights(disriminator_base)
    model.layers[model_names.index("classifier_base")].load_weights(classifier_base)
    model.layers[model_names.index("phase")].load_weights(phase)
    model.layers[model_names.index("amplitude")].load_weights(amplitude)
    model.layers[model_names.index("tracenet")].load_weights(tracenet)
    
    # print summary
    model.summary()
    
    # save model
    model.save(modelfile)
    
    if RUNNING_IN_COLAB:
        !cp models /content/drive/MyDrive/DeepLearningFinalProject/
    

In [ ]:
input = layers.Input(shape=(1, 300))  # Returns a placeholder tensor
classifer_feature_vec = pn.TraceClassifierBase(name = "classifier_base")(input)
discriminator_feature_vec = pn.TraceDiscriminatorBase(name = "discriminator_base")(input)
trace_output = pn.TraceClassifierDiscriminatorHead(name = "tracenet")([discriminator_feature_vec, classifer_feature_vec])

x_trace, y_trace, y_phase, y_amp, y_pileup = GetData()
# create model
model = models.Model(inputs=input, outputs=trace_output)
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
model_names = [model.layers[i].name for i in range(len(model.layers))]
                          
# print summary
model.summary()                                                         
# Train Discriminator Head
history = model.fit(x_trace, y_trace, epochs=epochs, batch_size=batchsize, validation_split=validation_split, verbose=1)
# save weights
model.layers[model_names.index("discriminator_base")].save_weights('weights/full_transfer_discriminator_base_weights_contingent.h5')
model.layers[model_names.index("classifier_base")].save_weights('weights/full_transfer_classifier_base_weights_contingent.h5')
model.layers[model_names.index("tracenet")].save_weights('weights/full_transfer_tracenet_weights_contingent.h5')

# save history
pd.DataFrame(history.history, index=history.epoch, columns=history.history.keys()).to_hdf('history/full_transfer_history_contingent.h5', key="hist")
# if RUNNING_IN_COLAB:
!cp -rf weights /content/drive/MyDrive/DeepLearningFinalProject/
!cp -rf history /content/drive/MyDrive/DeepLearningFinalProject/

In [ ]:
input = layers.Input(shape=(1, 300))  # Returns a placeholder tensor
classifer_feature_vec = pn.TraceClassifierBase(name = "classifier_base")(input)
discriminator_feature_vec = pn.TraceDiscriminatorBase(name = "discriminator_base")(input)
trace_output = pn.TraceClassifierDiscriminatorHead(name = "tracenet")([discriminator_feature_vec, classifer_feature_vec])
phase_output = pn.TracePhaseRegressor(name="phase")(trace_output)
x_trace, y_trace, y_phase, y_amp, y_pileup = GetData()
# create model
model = models.Model(inputs=input, outputs=phase_output)
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
model_names = [model.layers[i].name for i in range(len(model.layers))]
# load weights
model.layers[model_names.index("discriminator_base")].load_weights('weights/full_transfer_discriminator_base_weights_contingent.h5')
model.layers[model_names.index("classifier_base")].load_weights('weights/full_transfer_classifier_base_weights_contingent.h5')
model.layers[model_names.index("tracenet")].load_weights('weights/full_transfer_tracenet_weights_contingent.h5')
model.layers[model_names.index("discriminator_base")].trainable = False
model.layers[model_names.index("classifier_base")].trainable = False
model.layers[model_names.index("tracenet")].trainable = False
# print summary
model.summary()                                                         
# Train Discriminator Head
history = model.fit(x_trace, y_phase, epochs=epochs, batch_size=batchsize, validation_split=validation_split, verbose=1)
# save weights
model.layers[model_names.index("phase")].save_weights('weights/full_transfer_phase_weights_contingent.h5')

pd.DataFrame(history.history, index=history.epoch, columns=history.history.keys()).to_hdf('history/full_transfer_phase_history_contingent.h5', key="hist")
# if RUNNING_IN_COLAB:
!cp -rf weights /content/drive/MyDrive/DeepLearningFinalProject/
!cp -rf history /content/drive/MyDrive/DeepLearningFinalProject/

In [ ]:
input = layers.Input(shape=(1, 300))  # Returns a placeholder tensor
classifer_feature_vec = pn.TraceClassifierBase(name = "classifier_base")(input)
discriminator_feature_vec = pn.TraceDiscriminatorBase(name = "discriminator_base")(input)
trace_output = pn.TraceClassifierDiscriminatorHead(name = "tracenet")([discriminator_feature_vec, classifer_feature_vec])
amplitude_output = pn.TraceAmplitudeRegressor(name ="amplitude")(trace_output)

x_trace, y_trace, y_phase, y_amp, y_pileup = GetData()
# create model
model = models.Model(inputs=input, outputs=amplitude_output)
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
model_names = [model.layers[i].name for i in range(len(model.layers))]
# load weights
model.layers[model_names.index("discriminator_base")].load_weights('weights/full_transfer_discriminator_base_weights_contingent.h5')
model.layers[model_names.index("classifier_base")].load_weights('weights/full_transfer_classifier_base_weights_contingent.h5')
model.layers[model_names.index("tracenet")].load_weights('weights/full_transfer_tracenet_weights_contingent.h5')
model.layers[model_names.index("discriminator_base")].trainable = False
model.layers[model_names.index("classifier_base")].trainable = False
model.layers[model_names.index("tracenet")].trainable = False
                      
# print summary
model.summary()                                                         
# Train Discriminator Head
history = model.fit(x_trace, y_amp, epochs=epochs, batch_size=batchsize, validation_split=validation_split, verbose=1)
# save weights
model.layers[model_names.index("amplitude")].save_weights('weights/full_transfer_amplitude_weights_contingent.h5')

pd.DataFrame(history.history, index=history.epoch, columns=history.history.keys()).to_hdf('history/full_transfer_amplitude_history_contingent.h5', key="hist")
# if RUNNING_IN_COLAB:
!cp -rf weights /content/drive/MyDrive/DeepLearningFinalProject/
!cp -rf history /content/drive/MyDrive/DeepLearningFinalProject/

In [4]:
model = models.Model(inputs=input, outputs=[trace_output, phase_output, amplitude_output])
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

In [5]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1, 300)]     0           []                               
                                                                                                  
 discriminator_base (TraceDiscr  (None, 2, 300)      1893858     ['input_1[0][0]']                
 iminatorBase)                                                                                    
                                                                                                  
 classifier_base (TraceClassifi  (None, 1, 300)      244456      ['input_1[0][0]']                
 erBase)                                                                                          
                                                                                              

## **Sub modules training**
- ✅ Discrminator
- ✅ Classifier
- ✅ Phase 
- ✅ Amplitude

# __Discriminator__ (Done)

In [ ]:
# Get Data
x_trace, y_trace, y_phase, y_amp, y_pileup = GetData()
# Create model
model = keras.Sequential([
    pn.TraceDiscriminatorBase(name="discriminator_base"),
    pn.TraceDiscriminatorHead(name="discriminator_head")
])
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

# Train model
history = model.fit(x_trace, y_trace, epochs=epochs, batch_size=batchsize, validation_split=validation_split, verbose=1)
model.layers[0].save_weights('weights/individual_discriminator_base_weights.h5')
model.layers[1].save_weights('weights/individual_discriminator_head_weights.h5')
# Save history
pd.DataFrame(history.history, index=history.epoch, columns=history.history.keys()).to_hdf('history/individual_discriminator_history.h5', key="hist")

if RUNNING_IN_COLAB:
    !cp -rf weights /content/drive/MyDrive/DeepLearningFinalProject/
    !cp -rf history /content/drive/MyDrive/DeepLearningFinalProject/

# __Classifier__ (Done)

In [ ]:
x_trace, y_trace, y_phase, y_amp, y_pileup = GetData()
# Create model
model = keras.Sequential([
    pn.TraceClassifierBase(name="classifier_base"),
    pn.TraceClassifierHead(name="classifier_head")
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model
history = model.fit(x_trace, y_pileup, epochs=epochs, batch_size=batchsize, validation_split=validation_split, verbose=1)
model.layers[0].save_weights('weights/individual_classifier_base_weights.h5')
model.layers[1].save_weights('weights/individual_classifier_head_weights.h5')
# Save history
pd.DataFrame(history.history, index=history.epoch, columns=history.history.keys()).to_hdf('history/individual_classifier_history.h5', key="hist")

if RUNNING_IN_COLAB:
    !cp -rf weights /content/drive/MyDrive/DeepLearningFinalProject/
    !cp -rf history /content/drive/MyDrive/DeepLearningFinalProject/

# __Amplitude__ (Done)

In [ ]:
x_trace, y_trace, y_phase, y_amp, y_pileup = GetData()
# Create model
model = keras.Sequential([
    pn.TraceAmplitudeRegressor(name = "amplitude")
])
model.compile(optimizer='adam', loss="mse", metrics=['accuracy'])

# Train model
history = model.fit(x_trace, y_amp, epochs=epochs, batch_size=batchsize, validation_split=validation_split, verbose=1)
model.layers[0].save_weights('weights/individual_amplitude_weights.h5')
pd.DataFrame(history.history, index=history.epoch, columns=history.history.keys()).to_hdf('history/individual_amplitude_history.h5', key="hist")

if RUNNING_IN_COLAB:
    !cp -rf weights /content/drive/MyDrive/DeepLearningFinalProject/
    !cp -rf history /content/drive/MyDrive/DeepLearningFinalProject/

# __Phase__ (Done)

In [ ]:
x_trace, y_trace, y_phase, y_amp, y_pileup = GetData()
# Create model
model = keras.Sequential([
    pn.TracePhaseRegressor(name = "phase")
])
model.compile(optimizer='adam', loss="mse", metrics=['accuracy'])

# Train model
history = model.fit(x_trace, y_phase, epochs=epochs, batch_size=batchsize, validation_split=validation_split, verbose=1)
model.layers[0].save_weights('weights/individual_phase_weights.h5')
pd.DataFrame(history.history, index=history.epoch, columns=history.history.keys()).to_hdf('history/individual_phase_history.h5', key="hist")

if RUNNING_IN_COLAB:
    !cp -rf weights /content/drive/MyDrive/DeepLearningFinalProject/
    !cp -rf history /content/drive/MyDrive/DeepLearningFinalProject/

# **__Transfer Learning__**
_____
* ✅ Transfer 1 (Full)
* ✅ Transfer 2 (Trace) 

In [4]:
input = layers.Input(shape=(1, 300))  # Returns a placeholder tensor
classifer_feature_vec = pn.TraceClassifierBase(name = "classifier_base")(input)
discriminator_feature_vec = pn.TraceDiscriminatorBase(name = "discriminator_base")(input)
trace_output = pn.TraceClassifierDiscriminatorHead(name = "tracenet")([discriminator_feature_vec, classifer_feature_vec])
phase_output = pn.TracePhaseRegressor(name="phase")(trace_output)
amplitude_output = pn.TraceAmplitudeRegressor(name ="amplitude")(trace_output)
classifier_output = pn.TraceClassifierHead(name="classifier_head")(classifer_feature_vec)

2023-05-14 01:29:07.914648: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/tmengel/root/lib
2023-05-14 01:29:07.915640: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-05-14 01:29:07.915927: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tmengel): /proc/driver/nvidia/version does not exist
2023-05-14 01:29:07.922093: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


AttributeError: Exception encountered when calling layer "phase" (type TracePhaseRegressor).

in user code:

    File "/home/tmengel/DeepLearning/Deep-Learning/finalProject/pulsenet.py", line 301, in call  *
        x = self.MaxPooling1D1(x)

    AttributeError: 'TracePhaseRegressor' object has no attribute 'MaxPooling1D1'


Call arguments received by layer "phase" (type TracePhaseRegressor):
  • inputs=tf.Tensor(shape=(None, 2, 300), dtype=float32)

## __Transfer 1 (Direct)__ (Done)
* ✅ Transfer trace, phase and classifier onto new head
* ✅ Fine-tune model

#### Transfer

In [ ]:
x_trace, y_trace, y_phase, y_amp, y_pileup = GetData()
# create model
model = models.Model(inputs=input, outputs=[trace_output, phase_output, amplitude_output])
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
model_names = [model.layers[i].name for i in range(len(model.layers))]
# load weights
model.layers[model_names.index("discriminator_base")].load_weights('weights/individual_discriminator_base_weights.h5')
model.layers[model_names.index("classifier_base")].load_weights('weights/individual_classifier_base_weights.h5')
model.layers[model_names.index("phase")].load_weights('weights/individual_phase_weights.h5')
model.layers[model_names.index("amplitude")].load_weights('weights/individual_amplitude_weights.h5')
model.layers[model_names.index("discriminator_base")].trainable = False
model.layers[model_names.index("classifier_base")].trainable = False
model.layers[model_names.index("phase")].trainable = False
model.layers[model_names.index("amplitude")].trainable = False                           
# print summary
model.summary()                                                         
# Train Discriminator Head
history = model.fit(x_trace, [y_trace, y_phase, y_amp], epochs=epochs, batch_size=batchsize, validation_split=validation_split, verbose=1)
# save weights
model.layers[model_names.index("tracenet")].save_weights('weights/full_transfer_tracenet_weights.h5')
# save model
SaveModel(disciminator_base='weights/individual_discriminator_base_weights.h5',
          classifier_base='weights/individual_classifier_base_weights.h5',
          phase='weights/individual_phase_weights.h5',
          amplitude='weights/individual_amplitude_weights.h5',
          tracenet='weights/full_transfer_tracenet_weights.h5',
          modelfile='models/full_transfer_model.h5')
# save history
pd.DataFrame(history.history, index=history.epoch, columns=history.history.keys()).to_hdf('history/full_transfer_history.h5', key="hist")
if RUNNING_IN_COLAB:
    !cp -rf weights /content/drive/MyDrive/DeepLearningFinalProject/
    !cp -rf history /content/drive/MyDrive/DeepLearningFinalProject/

#### Fine-tune

In [ ]:
x_trace, y_trace, y_phase, y_amp, y_pileup = GetData()
# create model
model = models.Model(inputs=input, outputs=[trace_output, phase_output, amplitude_output])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss=['mse', 'mse', 'mse'], metrics=['accuracy'])
model_names = [model.layers[i].name for i in range(len(model.layers))]
# load weights
model.layers[model_names.index("discriminator_base")].load_weights('weights/individual_discriminator_base_weights.h5')
model.layers[model_names.index("classifier_base")].load_weights('weights/individual_classifier_base_weights.h5')
model.layers[model_names.index("phase")].load_weights('weights/individual_phase_weights.h5')
model.layers[model_names.index("amplitude")].load_weights('weights/individual_amplitude_weights.h5')
model.layers[model_names.index("tracenet")].load_weights('weights/full_transfer_tracenet_weights.h5')                
# print summary
model.summary()                                                         
# Train Discriminator Head
history = model.fit(x_trace, [y_trace, y_phase, y_amp], epochs=epochs, batch_size=batchsize, validation_split=validation_split, verbose=1)
# save weights
model.layers[model_names.index("discriminator_base")].save_weights('weights/full_transfer_fine_tuned_discriminator_base_weights.h5')
model.layers[model_names.index("classifier_base")].save_weights('weights/full_transfer_fine_tuned_classifier_base_weights.h5')
model.layers[model_names.index("phase")].save_weights('weights/full_transfer_fine_tuned_phase_weights.h5')
model.layers[model_names.index("amplitude")].save_weights('weights/full_transfer_fine_tuned_amplitude_weights.h5')
model.layers[model_names.index("tracenet")].save_weights('weights/full_transfer_fine_tuned_tracenet_weights.h5')
# save model
SaveModel(disciminator_base= 'weights/full_transfer_fine_tuned_discriminator_base_weights.h5',
          classifier_base= 'weights/full_transfer_fine_tuned_classifier_base_weights.h5',
          phase= 'weights/full_transfer_fine_tuned_phase_weights.h5',
          amplitude= 'weights/full_transfer_fine_tuned_amplitude_weights.h5',
          tracenet= 'weights/full_transfer_fine_tuned_tracenet_weights.h5',
          modelfile='models/full_transfer_fine_tuned_model.h5')
# save history
pd.DataFrame(history.history, index=history.epoch, columns=history.history.keys()).to_hdf('history/full_transfer_fine_tuning_history.h5', key="hist")
if RUNNING_IN_COLAB:
    !cp -rf weights /content/drive/MyDrive/DeepLearningFinalProject/
    !cp -rf history /content/drive/MyDrive/DeepLearningFinalProject/

# __Transfer 2 (Long)__ (Done)
* ✅ Transfer Trace onto head
* ✅ Transfer amplitude and phase onto head
* ✅ Fine tune


#### Transfer Trace

In [ ]:
x_trace, y_trace, y_phase, y_amp, y_pileup = GetData()
epochs = 200
# create model
model = models.Model(inputs=input, outputs=trace_output)
model.compile(optimizer='adam', loss="mse", metrics=['accuracy'])
model_names = [model.layers[i].name for i in range(len(model.layers))]
# load weights
model.layers[model_names.index("discriminator_base")].load_weights('weights/individual_discriminator_base_weights.h5')
model.layers[model_names.index("classifier_base")].load_weights('weights/individual_classifier_base_weights.h5')
# freeze layers
model.layers[model_names.index("discriminator_base")].trainable = False
model.layers[model_names.index("classifier_base")].trainable = False                         
# print summary
model.summary()                                                         
# Train Discriminator Head
history = model.fit(x_trace,y_trace, epochs=epochs, batch_size=batchsize, validation_split=validation_split, verbose=1)
# save weights
model.layers[model_names.index("tracenet")].save_weights('weights/trace_transfer_tracenet_weights.h5')
# save model
SaveModel(disciminator_base='weights/individual_discriminator_base_weights.h5',
            classifier_base='weights/individual_classifier_base_weights.h5',
            phase='weights/individual_phase_weights.h5',
            amplitude='weights/individual_amplitude_weights.h5',
            tracenet='weights/trace_transfer_tracenet_weights.h5',
            modelfile='models/trace_transfer_model.h5')

# save history
pd.DataFrame(history.history, index=history.epoch, columns=history.history.keys()).to_hdf('history/trace_transfer_history.h5', key="hist")
if RUNNING_IN_COLAB:
    !cp -rf weights /content/drive/MyDrive/DeepLearningFinalProject/
    !cp -rf history /content/drive/MyDrive/DeepLearningFinalProject/

#### Fine Tune Phase

In [ ]:
x_trace, y_trace, y_phase, y_amp, y_pileup = GetData()
epochs = 200
# create model
model = models.Model(inputs=input, outputs=phase_output)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss="mse", metrics=['accuracy'])
model_names = [model.layers[i].name for i in range(len(model.layers))]
# load weights
model.layers[model_names.index("discriminator_base")].load_weights('weights/individual_discriminator_base_weights.h5')
model.layers[model_names.index("classifier_base")].load_weights('weights/individual_classifier_base_weights.h5')
model.layers[model_names.index("phase")].load_weights('weights/individual_phase_weights.h5')
model.layers[model_names.index("tracenet")].load_weights('weights/trace_transfer_tracenet_weights.h5')
# freeze layers
model.layers[model_names.index("discriminator_base")].trainable = False
model.layers[model_names.index("classifier_base")].trainable = False
model.layers[model_names.index("tracenet")].trainable = False
# print summary
model.summary()                                                         
# Train Discriminator Head
history = model.fit(x_trace, y_phase, epochs=epochs, batch_size=batchsize, validation_split=validation_split, verbose=1)
# save weights
model.layers[model_names.index("phase")].save_weights('weights/trace_transfer_fine_tuned_phase_weights.h5')
# save history
pd.DataFrame(history.history, index=history.epoch, columns=history.history.keys()).to_hdf('history/trace_transfer_phase_fine_tuning_history.h5', key="hist")
if RUNNING_IN_COLAB:
    !cp -rf weights /content/drive/MyDrive/DeepLearningFinalProject/
    !cp -rf history /content/drive/MyDrive/DeepLearningFinalProject/

###  Fine Tune Amplitude

In [ ]:
x_trace, y_trace, y_phase, y_amp, y_pileup = GetData()
epochs = 200
# create model
model = models.Model(inputs=input, outputs=amplitude_output)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss="mse", metrics=['accuracy'])
model_names = [model.layers[i].name for i in range(len(model.layers))]
# load weights
model.layers[model_names.index("discriminator_base")].load_weights('weights/individual_discriminator_base_weights.h5')
model.layers[model_names.index("classifier_base")].load_weights('weights/individual_classifier_base_weights.h5')
model.layers[model_names.index("amplitude")].load_weights('weights/individual_amplitude_weights.h5')
model.layers[model_names.index("tracenet")].load_weights('weights/trace_transfer_tracenet_weights.h5')
# freeze layers
model.layers[model_names.index("discriminator_base")].trainable = False
model.layers[model_names.index("classifier_base")].trainable = False
model.layers[model_names.index("tracenet")].trainable = False
# print summary
model.summary()                                                         
# Train Discriminator Head
history = model.fit(x_trace, y_amp, epochs=epochs, batch_size=batchsize, validation_split=validation_split, verbose=1)
# save weights
model.layers[model_names.index("amplitude")].save_weights('weights/trace_transfer_fine_tuned_amplitude_weights.h5')
# save history
pd.DataFrame(history.history, index=history.epoch, columns=history.history.keys()).to_hdf('history/trace_transfer_amplitude_fine_tuning_history.h5', key="hist")
if RUNNING_IN_COLAB:
    !cp -rf weights /content/drive/MyDrive/DeepLearningFinalProject/
    !cp -rf history /content/drive/MyDrive/DeepLearningFinalProject/

Save model

In [ ]:
SaveModel(disciminator_base='weights/individual_discriminator_base_weights.h5',
            classifier_base='weights/individual_classifier_base_weights.h5',
            phase='weights/trace_transfer_fine_tuned_phase_weights.h5',
            amplitude='weights/trace_transfer_fine_tuned_amplitude_weights.h5',
            tracenet='weights/trace_transfer_tracenet_weights.h5',
            modelfile='models/trace_transfer_fine_tuned_model.h5')

# __Control__
* ✅ Train whole network at once

In [ ]:
x_trace, y_trace, y_phase, y_amp, y_pileup = GetData()
epochs = 1000
# create model
model = models.Model(inputs=input, outputs=[trace_output, phase_output, amplitude_output])
model.compile(optimizer="adam",loss="mse", metrics=['accuracy'])
model_names = [model.layers[i].name for i in range(len(model.layers))]
# print summary
model.summary()
# train control model
history = model.fit(x_trace, [y_trace, y_phase, y_amp], epochs=epochs, batch_size=batchsize, validation_split=validation_split, verbose=1)
# save weights
model.layers[model_names.index("discriminator_base")].save_weights('weights/control_discriminator_base_weights.h5')
model.layers[model_names.index("classifier_base")].save_weights('weights/control_classifier_base_weights.h5')
model.layers[model_names.index("phase")].save_weights('weights/control_phase_weights.h5')
model.layers[model_names.index("amplitude")].save_weights('weights/control_amplitude_weights.h5')
model.layers[model_names.index("tracenet")].save_weights('weights/control_tracenet_weights.h5')
# save model
SaveModel(disciminator_base='weights/control_discriminator_base_weights.h5',
            classifier_base='weights/control_classifier_base_weights.h5',
            phase='weights/control_phase_weights.h5',
            amplitude='weights/control_amplitude_weights.h5',
            tracenet='weights/control_tracenet_weights.h5',
            modelfile='models/control_model.h5')

# save history
pd.DataFrame(history.history, index=history.epoch, columns=history.history.keys()).to_hdf('history/control_history.h5', key="hist")
if RUNNING_IN_COLAB:
    !cp -rf weights /content/drive/MyDrive/DeepLearningFinalProject/
    !cp -rf history /content/drive/MyDrive/DeepLearningFinalProject/
    

# __Sequential Training__
* ✅ Train whole model on trace
* ✅ Train whole model on phase
* ✅ Train whole model on amplitude

In [ ]:
input = layers.Input(shape=(1, 300))  # Returns a placeholder tensor
discriminator_feature_vec = pn.TraceDiscriminatorBase(name = "discriminator_base")(input)
trace_output = pn.TraceDiscriminatorHead(name = "discriminator_head")(discriminator_feature_vec)
phase_output = pn.TracePhaseRegressor(name="phase")(trace_output)
amplitude_output = pn.TraceAmplitudeRegressor(name ="amplitude")(trace_output)

###  Train Trace

In [ ]:
# load new data
epochs = 700
x_trace, y_trace, y_phase, y_amp, y_pileup = GetData()

# Train Discriminator
# create model
model = models.Model(inputs=input, outputs=trace_output)
model.compile(optimizer="adam", loss='mse', metrics=['accuracy'])
model_names = [model.layers[i].name for i in range(len(model.layers))]
# print summary
model.summary()
# train model
history = model.fit(x_trace, y_trace, epochs=epochs, batch_size=batchsize, validation_split=validation_split, verbose=1)
# save weights
model.layers[model_names.index("discriminator_base")].save_weights('weights/consecutive_discriminator_base_weights.h5')
model.layers[model_names.index("discriminator_head")].save_weights('weights/consecutive_discriminator_head_weights.h5')
# save history
pd.DataFrame(history.history, index=history.epoch, columns=history.history.keys()).to_hdf('history/consecutive_discriminator_history.h5', key="hist")
if RUNNING_IN_COLAB:
    !cp -rf weights /content/drive/MyDrive/DeepLearningFinalProject/
    !cp -rf history /content/drive/MyDrive/DeepLearningFinalProject/

###  Train Phase

In [ ]:
epochs = 700
x_trace, y_trace, y_phase, y_amp, y_pileup = GetData()
model = models.Model(inputs=input, outputs=phase_output)
model.compile(optimizer="adam", loss='mse', metrics=['accuracy'])
model_names = [model.layers[i].name for i in range(len(model.layers))]
model.layers[model_names.index("discriminator_base")].load_weights('weights/consecutive_discriminator_base_weights.h5')
model.layers[model_names.index("discriminator_head")].load_weights('weights/consecutive_discriminator_head_weights.h5')
model.layers[model_names.index("discriminator_base")].trainable = False
model.layers[model_names.index("discriminator_head")].trainable = False
# print summary
model.summary()
history = model.fit(x_trace, y_phase, epochs=epochs, batch_size=batchsize, validation_split=validation_split, verbose=1)
# save weights
model.layers[model_names.index("phase")].save_weights('weights/consecutive_phase_weights.h5')
# save history
pd.DataFrame(history.history, index=history.epoch, columns=history.history.keys()).to_hdf('history/consecutive_phase_history.h5', key="hist")
if RUNNING_IN_COLAB:
    !cp -rf weights /content/drive/MyDrive/DeepLearningFinalProject/
    !cp -rf history /content/drive/MyDrive/DeepLearningFinalProject/

### Train Amplitude

In [ ]:
epochs = 700
x_trace, y_trace, y_phase, y_amp, y_pileup = GetData()
model = models.Model(inputs=input, outputs=amplitude_output)
model.compile(optimizer="adam", loss='mse', metrics=['accuracy'])
model_names = [model.layers[i].name for i in range(len(model.layers))]
model.layers[model_names.index("discriminator_base")].load_weights('weights/consecutive_discriminator_base_weights.h5')
model.layers[model_names.index("discriminator_head")].load_weights('weights/consecutive_discriminator_head_weights.h5')
model.layers[model_names.index("discriminator_base")].trainable = False
model.layers[model_names.index("discriminator_head")].trainable = False
# print summary
model.summary()
history = model.fit(x_trace, y_amp, epochs=epochs, batch_size=batchsize, validation_split=validation_split, verbose=1)
# save weights
model.layers[model_names.index("amplitude")].save_weights('weights/consecutive_amplitude_weights.h5')
# save history
pd.DataFrame(history.history, index=history.epoch, columns=history.history.keys()).to_hdf('history/consecutive_amplitude_history.h5', key="hist")
if RUNNING_IN_COLAB:
    !cp -rf weights /content/drive/MyDrive/DeepLearningFinalProject/
    !cp -rf history /content/drive/MyDrive/DeepLearningFinalProject/

### Save Model

In [ ]:
input = layers.Input(shape=(1, 300))  # Returns a placeholder tensor
discriminator_feature_vec = pn.TraceDiscriminatorBase(name = "discriminator_base")(input)
trace_output = pn.TraceDiscriminatorHead(name = "discriminator_head")(discriminator_feature_vec)
phase_output = pn.TracePhaseRegressor(name="phase")(trace_output)
amplitude_output = pn.TraceAmplitudeRegressor(name ="amplitude")(trace_output)

model = models.Model(inputs=input, outputs=[trace_output, phase_output, amplitude_output])
model.compile(optimizer="adam", loss='mse', metrics=['accuracy'])
model_names = [model.layers[i].name for i in range(len(model.layers))]
model.layers[model_names.index("discriminator_base")].load_weights('weights/consecutive_discriminator_base_weights.h5')
model.layers[model_names.index("discriminator_head")].load_weights('weights/consecutive_discriminator_head_weights.h5')
model.layers[model_names.index("phase_regressor")].load_weights('weights/consecutive_phase_weights.h5')
model.layers[model_names.index("amplitude_regressor")].load_weights('weights/consecutive_amplitude_weights.h5')

# print summary
model.summary()
# save model
model.save("models/consecutive_model.h5")
